In [71]:
import numpy as np
import matplotlib as plt
from itertools import permutations

from keras.preprocessing.image import load_img, img_to_array
import os

import xml.etree.ElementTree as ElementTree

from PIL import Image

In [72]:
class Bbox:
    def __init__(self, xmin, ymin, xmax, ymax):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax


In [73]:
def generate_anchor(base_width=16, base_height=16, anchor_ratios=[0.5, 1, 2], anchor_scales=[1, 2, 4, 6]):
    base_anchor = np.array([base_width, base_height])
    
    ratio_permute = np.array(list(permutations(anchor_ratios, 2))) * base_width
    base_ratio_anchor = np.vstack((ratio_permute, base_anchor))

    scaled_anchor = []

    for i in anchor_scales:
        temp = base_ratio_anchor * i
        scaled_anchor.append(temp)

    scaled_anchor = np.vstack(scaled_anchor)
    scaled_anchor = np.hstack((np.ones(scaled_anchor.shape), scaled_anchor))
generate_anchor()

In [78]:
def load_image(base_dir='/myssddisk/Programming/Git/4641-Project/datasets/imagenet/apple-n07739344/img/'):
    
    all_img = []

    print("called")
    for file in os.listdir(base_dir):
        filedir = base_dir
        filename = os.fsdecode(file)
        filepath = filedir + filename
        
        try:
            img = load_img(filepath, target_size=(299, 299, 3))
            img = img_to_array(img)

            all_img.append(img)
        except:
            print("something went wrong")
            #os.remove(filepath)

    all_img = np.array(all_img)
    return all_img


In [79]:
def load_annotation(base_dir='/myssddisk/Programming/Git/4641-Project/datasets/imagenet/apple-n07739344/bbox/'):

    wnid_annotation = {
        "wnid": None,
        "count": 0,
        "annotations": []
    }

    for file in os.listdir(base_dir):

        subwnid_annotation = {
            "subwnid": None,
            "img": None,
            "count": 0,
            "bboxes": []
        }

        filedir = base_dir
        filename = os.fsdecode(file)
        filepath = filedir + filename

        tree = ElementTree.parse(filepath)
        root = tree.getroot()

        # Setting the count number for wnid_annotation
        wnid_annotation["count"] += 1
        wnid_annotation["wnid"] = root.find('folder').text

        # Setting the subwnid for the current picture
        subwnid = root.find('filename').text
        dash_index = subwnid.index('_')
        subwnid = subwnid[dash_index + 1: ]
        subwnid_annotation["subwnid"] = subwnid
    
        for boxes in root.iter('object'):

            for box in boxes.findall("bndbox"):
                xmin = int(box.find("xmin").text)
                xmax = int(box.find("xmax").text)
                ymin = int(box.find("ymin").text)
                ymax = int(box.find("ymax").text)

                temp_bbox = Bbox(xmin, xmax, ymin, ymax)
                subwnid_annotation["bboxes"].append(temp_bbox)
                subwnid_annotation["count"] += 1
        
        wnid_annotation["annotations"].append(subwnid_annotation)
    
    return wnid_annotation
            


In [80]:
def load_data():
    all_annotations = load_annotation()
    all_img = load_image()

In [81]:
load_data()

called
